## lab-cleaning-categorical-data

In [1]:
#1. Import the necessary libraries if you are starting a new notebook. 
#Using the same data as the previous lab: we_fn_use_c_marketing_customer_value_analysis.csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

customer_df = pd.read_csv('we_fn_use_c_marketing_customer_value_analysis.csv')


In [2]:
#2. Find all of the categorical data. Save it in a categorical_df variable.
categorical_df = customer_df.select_dtypes(['object'])

In [3]:
#3.Check for NaN values.
customer_df.isna().sum()

Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
EmploymentStatus                 0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
dtype: int64

In [7]:
#Additional data step: Renaming all the columns to snakecase
cols=[]
for c in categorical_df.columns:
    cols.append(c.lower())
categorical_df.columns=cols
categorical_df.columns = categorical_df.columns.str.replace(' ', '_')
# changing the name of employmentstatus -> employment_status
categorical_df = categorical_df.rename(columns={'employmentstatus':'employment_status'})
# changing the effective date
categorical_df['effective_to_date'] = pd.to_datetime(categorical_df['effective_to_date'], 
                                  dayfirst=True).dt.strftime('%d/%m/%Y')

In [8]:
#4.Check all unique values of columns
for col in categorical_df:
  print(col,categorical_df[col].unique())

customer ['BU79786' 'QZ44356' 'AI49188' ... 'TD14365' 'UP19263' 'Y167826']
state ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response ['No' 'Yes']
coverage ['Basic' 'Extended' 'Premium']
education ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
effective_to_date ['24/02/2011' '31/01/2011' '19/02/2011' '20/01/2011' '02/03/2011'
 '25/01/2011' '18/01/2011' '26/01/2011' '17/02/2011' '21/02/2011'
 '01/06/2011' '02/06/2011' '01/10/2011' '17/01/2011' '01/05/2011'
 '27/02/2011' '14/01/2011' '21/01/2011' '02/05/2011' '29/01/2011'
 '28/02/2011' '02/12/2011' '02/02/2011' '02/07/2011' '22/01/2011'
 '13/02/2011' '15/01/2011' '01/08/2011' '01/11/2011' '28/01/2011'
 '02/08/2011' '23/02/2011' '01/02/2011' '16/02/2011' '27/01/2011'
 '23/01/2011' '01/09/2011' '02/11/2011' '02/04/2011' '02/01/2011'
 '15/02/2011' '26/02/2011' '16/01/2011' '01/01/2011' '02/10/2011'
 '24/01/2011' '25/02/2011' '01/12/2011' '02/09/2011' '19/01/2011'
 '01/04/2011' '14/02/2011' '20/02/2011' '18/02/20

In [9]:
#5. Check dtypes. Do they all make sense as categorical data?
#Upon further looking at the data, I would consider change respose to numerical dtype rather than categorical. 
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   customer           9134 non-null   object
 1   state              9134 non-null   object
 2   response           9134 non-null   object
 3   coverage           9134 non-null   object
 4   education          9134 non-null   object
 5   effective_to_date  9134 non-null   object
 6   employment_status  9134 non-null   object
 7   gender             9134 non-null   object
 8   location_code      9134 non-null   object
 9   marital_status     9134 non-null   object
 10  policy_type        9134 non-null   object
 11  policy             9134 non-null   object
 12  renew_offer_type   9134 non-null   object
 13  sales_channel      9134 non-null   object
 14  vehicle_class      9134 non-null   object
 15  vehicle_size       9134 non-null   object
dtypes: object(16)
memory usage: 1.1+ MB


In [13]:
#6.Does any column contain alpha and numeric data? Decide how to clean it.
#I decided to drop Customer as they have mostly unique values, and wouldn't be beneficial to the evaluation of the data. 
categorical_df = categorical_df.drop(columns=['customer'])

In [14]:
#7. Would you choose to do anything else to clean or wrangle the categorical data? Comment your decisions.
#I would consider moving all of the divorced customers to single values
categorical_df['marital_status'].value_counts()

Married     5298
Single      2467
Divorced    1369
Name: marital_status, dtype: int64

In [16]:
categorical_df['marital_status'] = np.where(categorical_df['marital_status'].isin(['Divorced']) , 'Single', categorical_df['marital_status'])
categorical_df['marital_status'].value_counts()

Married    5298
Single     3836
Name: marital_status, dtype: int64

In [20]:
#Changing the yes to 1s, and No to 0s in the response column.
categorical_df['response'] = categorical_df['response'].apply(lambda x: 1 if x == 'Yes' else 0)

In [21]:
categorical_df.head()

,state,response,coverage,education,effective_to_date,employment_status,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Washington,0,Basic,Bachelor,24/02/2011,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,Arizona,0,Extended,Bachelor,31/01/2011,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,Nevada,0,Premium,Bachelor,19/02/2011,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,California,0,Basic,Bachelor,20/01/2011,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,Washington,0,Basic,Bachelor,02/03/2011,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


In [22]:
#8. Compare policy_type and policy. What information is contained in these columns. Can you identify what is important?
categorical_df['policy_type'].value_counts()

Personal Auto     6788
Corporate Auto    1968
Special Auto       378
Name: policy_type, dtype: int64

In [23]:
categorical_df['policy'].value_counts()

Personal L3     3426
Personal L2     2122
Personal L1     1240
Corporate L3    1014
Corporate L2     595
Corporate L1     359
Special L2       164
Special L3       148
Special L1        66
Name: policy, dtype: int64

In [ ]:
# I would consider dropping policy_type as there is more specified data keeping policy instead of policy-type.

In [24]:
#9. Check number of unique values in each column, can they be combined in any way to ease encoding? Comment your thoughts and make those changes.
for col in categorical_df:
  print(col,categorical_df[col].unique())

state ['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
response [0 1]
coverage ['Basic' 'Extended' 'Premium']
education ['Bachelor' 'College' 'Master' 'High School or Below' 'Doctor']
effective_to_date ['24/02/2011' '31/01/2011' '19/02/2011' '20/01/2011' '02/03/2011'
 '25/01/2011' '18/01/2011' '26/01/2011' '17/02/2011' '21/02/2011'
 '01/06/2011' '02/06/2011' '01/10/2011' '17/01/2011' '01/05/2011'
 '27/02/2011' '14/01/2011' '21/01/2011' '02/05/2011' '29/01/2011'
 '28/02/2011' '02/12/2011' '02/02/2011' '02/07/2011' '22/01/2011'
 '13/02/2011' '15/01/2011' '01/08/2011' '01/11/2011' '28/01/2011'
 '02/08/2011' '23/02/2011' '01/02/2011' '16/02/2011' '27/01/2011'
 '23/01/2011' '01/09/2011' '02/11/2011' '02/04/2011' '02/01/2011'
 '15/02/2011' '26/02/2011' '16/01/2011' '01/01/2011' '02/10/2011'
 '24/01/2011' '25/02/2011' '01/12/2011' '02/09/2011' '19/01/2011'
 '01/04/2011' '14/02/2011' '20/02/2011' '18/02/2011' '01/03/2011'
 '13/01/2011' '30/01/2011' '22/02/2011' '01/07/2011']
employment_

In [ ]:
#yeah there would no additional changes that I would make, as the values are distinct enough to the columns. 